In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.metrics import classification_report, accuracy_score

tf.__version__

'2.4.1'

In [2]:
# -------- TEST USER ----------- #

TEST_USER      = '001'

BASE_DIR       = '../'
IMG_DIR        = 'Spectrogram-Images/'
LOG_DIR        = 'Logs/'

USERS          = ['001', '002', '003', '004', '005', '006', '007']

# ------------------------------- Only Dynalic Gestures ------------------------------ #
GESTURES       = ['j', 'z', 'bad', 'deaf', 'fine', 'good', 'goodbye', 'hello', 'hungry',
                  'me', 'no', 'please', 'sorry', 'thankyou', 'yes', 'you']

BATCH_SIZE     = 32
IMG_LEN        = 160
IMG_SIZE       = (IMG_LEN, IMG_LEN)

# ------------- FOR THE GREATER GOOD :) ------------- #
TRAIN_LEN      = 960
TEST_LEN       = 160

EPOCHS         = 15
LEARNING_RATE  = 0.001

In [3]:
def load_data():
    X_train = np.zeros((TRAIN_LEN, IMG_LEN, IMG_LEN, 3))
    X_test = np.zeros((TEST_LEN, IMG_LEN, IMG_LEN, 3))
    y_train = np.zeros((TRAIN_LEN, 1))
    y_test = np.zeros((TEST_LEN, 1))
    
    train_count = 0
    test_count = 0
        
    for gesture in GESTURES:
        print('loading data for the ' + gesture + ' gesture ... ', end='')
        path = os.path.join(BASE_DIR, IMG_DIR, gesture)
        for filename in os.listdir(path):
            img = cv2.imread(os.path.join(path, filename))
            resized = cv2.resize(img, IMG_SIZE)
            if filename[1:4] != TEST_USER:
                X_train[train_count, :] = resized
                y_train[train_count, 0] = GESTURES.index(gesture)
                train_count = train_count + 1
            else:
                X_test[test_count, :] = resized
                y_test[test_count, 0] = GESTURES.index(gesture)
                test_count = test_count + 1
                
        print('√')
        
    return X_train, X_test, y_train, y_test

In [4]:
X_train, X_test, y_train, y_test = load_data()

loading data for the j gesture ... √
loading data for the z gesture ... √
loading data for the bad gesture ... √
loading data for the deaf gesture ... √
loading data for the fine gesture ... √
loading data for the good gesture ... √
loading data for the goodbye gesture ... √
loading data for the hello gesture ... √
loading data for the hungry gesture ... √
loading data for the me gesture ... √
loading data for the no gesture ... √
loading data for the please gesture ... √
loading data for the sorry gesture ... √
loading data for the thankyou gesture ... √
loading data for the yes gesture ... √
loading data for the you gesture ... √


In [5]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

In [6]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model.trainable = False

In [7]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(len(GESTURES))

In [8]:
def get_model():
    inputs = tf.keras.Input(shape=IMG_SHAPE)
    x = preprocess_input(inputs)
    x = base_model(x, training=False)
    x = global_average_layer(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model

In [ ]:
model = get_model()
history = model.fit(X_train, y_train, epochs=EPOCHS)

Epoch 1/15
30/30 [==============================] - 12s 337ms/step - loss: 3.1878 - accuracy: 0.0840
Epoch 2/15
30/30 [==============================] - 10s 334ms/step - loss: 2.4368 - accuracy: 0.1950
Epoch 3/15
30/30 [==============================] - 10s 331ms/step - loss: 2.2299 - accuracy: 0.2736
Epoch 4/15
30/30 [==============================] - 10s 331ms/step - loss: 2.0444 - accuracy: 0.3229
Epoch 5/15
30/30 [==============================] - 10s 334ms/step - loss: 1.9109 - accuracy: 0.3584
Epoch 6/15
30/30 [==============================] - 10s 331ms/step - loss: 1.7930 - accuracy: 0.4179
Epoch 7/15
30/30 [==============================] - 10s 333ms/step - loss: 1.7035 - accuracy: 0.4398
Epoch 8/15
30/30 [==============================] - 10s 332ms/step - loss: 1.5992 - accuracy: 0.4663
Epoch 9/15
30/30 [==============================] - 10s 330ms/step - loss: 1.5755 - accuracy: 0.4887
Epoch 10/15


In [ ]:
prob = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
y_pred = prob.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
print(classification_report(y_test.ravel(), y_pred))